A amostra de dados em anexo (info_transportes.csv), possui dados de um
aplicativo de transporte privado, cujas colunas são:
* DATA_INICIO (formato: "mm-dd-yyyy HH")
* DATA_FIM (formato: "mm-dd-yyyy HH")
* CATEGORIA
* LOCAL_INICIO
* LOCAL_FIM
* PROPOSITO
* DISTANCIA

<br>
Uma equipe está elaborando um modelo para compreender como os clientes
estão utilizando o aplicativo. Para isso, você precisa fornecer uma nova tabela
“info_corridas_do_dia", com dados agrupados pela data de início do transporte
utilizando a formatação “yyyy-MM-dd”, contendo as seguintes colunas:

| Nome da Coluna | Descrição |
   |---|---|
   | DT_REFE | Data de referência. |
   | QT_CORR | Quantidade de corridas. |
   | QT_CORR_NEG | Quantidade de corridas com a categoria “Negócio”. |
   | QT_CORR_PESS | Quantidade de corridas com a categoria “Pessoal”. |
   | VL_MAX_DIST | Maior distância percorrida por uma corrida. |
   | VL_MIN_DIST | Menor distância percorrida por uma corrida. |
   | VL_AVG_DIST | Média das distâncias percorridas. |
   | QT_CORR_REUNI | Quantidade de corridas com o propósito de "Reunião". |
   | QT_CORR_NAO_REUNI | Quantidade de corridas com o propósito declarado e diferente de "Reunião". |
   
<br>
Abaixo temos um exemplo de uma possível linha da tabela:

| DT_REFE | QT_CORR | QT_CORR_NEG | QT_CORR_PESS | VL_MAX_DIST | VL_MIN_DIST | VL_AVG_DIST | QT_CORR_REUNI | QT_CORR_NAO_REUNI |
   |---|---|---|---|---|---|---|---|---|
   | 2022-01-01 | 20 | 12 | 8 | 2.2 | 0.7 | 1.1 | 6 | 10 |

O código que você irá elaborar para criar a tabela (ou dataframe spark/pandas)
deverá ser escrito em uma das seguintes linguagens: SQL ou Python (pyspark
e/ou pandas). Caso decida por SQL, considere o nome da tabela como "
info_transportes".

<h3>Solução 1 - Exploração de Dados com Pandas</h3>

In [19]:
import pandas as pd
import pyodbc

In [10]:
df = pd.read_csv('\\projetos\info_transportes.csv', sep=';')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1153 entries, 0 to 1152
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   DATA_INICIO   1153 non-null   object
 1   DATA_FIM      1153 non-null   object
 2   CATEGORIA     1153 non-null   object
 3   LOCAL_INICIO  1153 non-null   object
 4   LOCAL_FIM     1153 non-null   object
 5   DISTANCIA     1153 non-null   int64 
 6   PROPOSITO     651 non-null    object
dtypes: int64(1), object(6)
memory usage: 63.2+ KB


In [13]:
df.head()

,DATA_INICIO,DATA_FIM,CATEGORIA,LOCAL_INICIO,LOCAL_FIM,DISTANCIA,PROPOSITO
0,01-01-2016 21:11,01-01-2016 21:17,Negocio,Fort Pierce,Fort Pierce,51,Alimentação
1,01-02-2016 01:25,01-02-2016 01:37,Negocio,Fort Pierce,Fort Pierce,5,NaN
2,01-02-2016 20:25,01-02-2016 20:38,Negocio,Fort Pierce,Fort Pierce,48,Entregas
3,01-05-2016 17:31,01-05-2016 17:45,Negocio,Fort Pierce,Fort Pierce,47,Reunião
4,01-06-2016 14:42,01-06-2016 15:49,Negocio,Fort Pierce,West Palm Beach,637,Visita ao cliente


In [21]:
conn_str = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=(localdb)\LocalDBDemo;'
    r'DATABASE=BD_FIN_PES;'
    r'UID=CBYK-81\lucas.castro;'
    r'PWD=@81@cbyk@;'
)

In [23]:
nome_tabela = 'info_transportes'

In [24]:
conn = pyodbc.connect(conn_str)

InterfaceError: ('28000', "[28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user 'CBYK-81\\lucas.castro'. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user 'CBYK-81\\lucas.castro'. (18456)")

<h3>Solução 2 - Manipulação de Dados no SQL Server</h3>